# Practica 6

En esta práctica entrenaremos un nuevo clasificador de audio. En la práctica 5
diseñamos un total de 85 características a partir del espectrograma. En esta práctica
dejaremos que sea la red neuronal la que aprenda directamente a extraer estas características
a partir del espectrograma mel. 

Comenzaremos por las funciones necesarias para preprocesar los audios. Esto es, pasar
de las muestras originales al espectrograma mel. 


## Preprocesado de audio

El preprocesado de audio debe obtener el espectrograma de cada audio a partir de las muestras


In [ ]:
import torch
from torch import nn
import librosa
import numpy as np
from utils import nextpow2
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader
from utils import detect_speech
import pytest
import UPVlog
notebook_filename="practica6"
my_logger=UPVlog.UPVlog(notebook_filename)

La función siguiente debe extraer el espectrograma a partir de un fichero de audio. Los pasos serán los siguientes:

1.  Cargar el fichero wav
2.  Recortar los silencios iniciales y finales con `detect_speech`
3.   Si la duracion del audio recortado es menor que 1.1 segundos. Añadiremos ceros al final hasta completar esa longitud. El objetivo es que el todos los audios preprocesados tengan la misma duración. En caso de que la duración del audio recortado sea mayor que 1.1 segundos recortaremos el audio quedandonos con los primeros 1.1 segundos.
4.    Calculo del espectrograma mel con los parametros indicados en la función



In [ ]:
def linear_normalize(array):
    min_val = np.min(array)
    max_val = np.max(array)
    normalized_array = (array - min_val) / (max_val - min_val + 1e-10)
    return normalized_array


def preprocess_audio(filename):
    
    #Cargamos audio y recortamos silencios
    
    audio_data, sample_rate = librosa.load(filename, sr=None)
    
    audio_mask = detect_speech(audio_data, sample_rate)    

    audio_trim = audio_data[audio_mask>0]
    
    if len(audio_trim)< 0.1 * sample_rate :# detect speech no ha ido bien, descartamos audio
        return None
    
    target_duration = 1.1 # para que todos los audios duren lo mismo
    target_length = round(target_duration * sample_rate)

    if len(audio_trim) > target_length:
    # si la duracion es mayor que target length recortamos y si es menor rellenamos con ceros
    # Dejar el resultado en la variable audio
        audio = audio_trim[:target_length] # dejar el resultado en esta variable
    else:
        padding = target_length - len(audio_trim)
        zeros = np.zeros(padding, dtype=audio_trim.dtype)
        audio = np.concatenate([audio_trim, zeros])
        #o esto:
        # audio = np.pad(audio_trim, (0, padding), mode='constant')
        # hace lo mismo q np.zeros y np.concentrate pero todo en una linea
    
    n_mels = 32
    frameDuration = 0.03 # 30 ms
    win_length = int(frameDuration*sample_rate)
    n_fft = 2**nextpow2(win_length)
    
    #Calcula el espectrograma mel con los parametros anteriores
    hop_length = win_length//2

    S = librosa.feature.melspectrogram(
        
        y = audio,
        sr = sample_rate,
        n_fft = n_fft,
        hop_length=hop_length,
        win_length=win_length,
        n_mels=n_mels,
        power=2.0 
    )

    # Convert to decibels (log scale)
    S_dB = np.log(S+1e-6)

    S_dB = linear_normalize(S_dB) # Ajusta las amplitudes entre [0-1]

    return S_dB



In [ ]:
# Comprobación
my_logger.test("preprocesado audio")
S_dB = preprocess_audio('siete.wav')
#S_dB = preprocess_audio('/srv/data/audios/base_datos_numeros/train/seis/2023_008_AB_seis_4_1.wav')
assert S_dB.shape == (32,74), "comprueba como calculas el espectrograma mel"
#assert S_dB[10,10]==pytest.approx(0.30, abs=1e-2)

print(S_dB.shape)

# Plot the Mel spectrogram
sample_rate = 16000
librosa.display.specshow(S_dB, sr=sample_rate,  hop_length=240, x_axis='time', y_axis='mel')
plt.title('Mel-frequency spectrogram')
plt.tight_layout()
my_logger.success("preprocesado audio",4)


## Create dataset

Ahora crearemos los datasets para realizar el entrenamiento de la red neuronal. 

En primer lugar vamos a 
preprocesar todos los audios de la base de datos. Obteniendo como resultado 
dos tensores con todas las imagenes de train y validacion respectivamente. 



In [ ]:
#localizacion de los wav
audios_train = '/srv/data/audios/base_datos_numeros/train'
audios_valid = '/srv/data/audios/base_datos_numeros/test/'

#las etiquetas son el nombre de la carpeta
map_labels = {'cero': 0,
              'uno': 1,
              'dos': 2,
              'tres': 3,
              'cuatro':4,
              'cinco':5,
              'seis':6,
              'siete':7,
              'ocho':8,
              'nueve':9}


Para crear los datasets vamos a emplear la funcion `create_dataset`. Los pasos que realiza son:
1. Crea una lista con todos los ficheros wav
2. Los guarda en una tabla de pandas (parecido a una hoja excel)
3. Crea columna `label`, a partir del nombre de la carpeta de cada audio
4. Convierte `label` a un entero con el diccionario `map_labels`.
5. Para cada fichero extrae el espectrograma con la funcion `preprocess_audio`.
6. Crea un tensor con los espectrogramas y un tensor con las etiquetas
7. Con la clase `TensorDataset` de Pytorch crea el dataset.

In [ ]:

def create_dataset(audios_folder):
    audio_files = pathlib.Path(audios_folder).rglob('*.wav')
    df = pd.DataFrame({'filename': list(audio_files)})
    df['label']=df.filename.apply(lambda x: str(x).split('/')[-2]) 
    df['y'] = df.label.map(map_labels)
    print("Number of  images  in dataset= ", len(df))
    #print(df.head())

    all_specgrams = []
    all_labels = []

    num_audios = len(df)

    for k in tqdm(range( num_audios)):
        row = df.iloc[k]
        features_v = preprocess_audio(row['filename'])
        if features_v is None:
            continue

        if features_v.shape[1] < 10:
            print("bad audio ", row['filename'])
            continue
        all_labels.append(row['y'])
        all_specgrams.append(features_v[np.newaxis,:,:])
        #if k > 10:
        #    break
    all_specgrams = np.array(all_specgrams)
    all_labels = np.array(all_labels)

    X_tensor = torch.tensor(all_specgrams, dtype=torch.float32)
    Y_tensor = torch.tensor(all_labels, dtype=torch.long)

    # Create TensorDataset
    dataset = TensorDataset(X_tensor, Y_tensor)

    return dataset
        

In [ ]:
# creamos tanto el dataset de train como el de validacion

train_dataset = create_dataset(audios_train)
val_dataset = create_dataset(audios_valid)

Antes de continuar es conveniente visualizar algunos ejemplos del datast para ver que todo es correcto

In [ ]:
def show_samples(dataset, nrows=9, ncols = 3):
    fig, ax = plt.subplots(nrows,ncols, figsize = ( ncols*4, nrows*4))
    axl = ax.flatten()
    
    num_samples = len(train_dataset)
    for k in range(len(axl)):
        sample_idx = np.random.randint(0,num_samples)
        spec, label = dataset[sample_idx]
        librosa.display.specshow(spec[0].numpy(), sr=16000, hop_length=240, x_axis='time', y_axis='mel',ax= axl[k])
        #axl[k].imshow(spec[0])
        axl[k].set_title(f"label {label}")
    plt.subplots_adjust(hspace=0.3, wspace=0.3)

show_samples(train_dataset)




## Network

En este apartado definiremos la red neuronal convolucional. Al igual que en la practica 5, nos definiremos un modulo auxiliar 
que se repite varias veces en la red en concreto el módulo `conv_layer`. Debe añadir los siguientes elementos:
* `nn.Conv2d`: Convolucion 2d, kernel de tamaño 3 y padding =1
* `nn.BatchNorm2d`: Similar al batch norm para el caso de imagenes
* `nn.ReLU`: No linealidad

Como verá en el codigo de abajo, opcionalmente este modulo tambien puede realizar max pooling.


In [ ]:
from torch import nn

class conv_layer(nn.Sequential):
    def __init__(self,input_feat, out_feat, max_pool = True):
        layers = []
        #Complete con vonv2d, batchnorm2d y relu añadiendo a la lista: layers.append(...)
        
        # 1) Convolución 2d, kernel=3, padding=1
        layers.append(nn.Conv2d(in_channels=input_feat,
                                out_channels=out_feat,
                                kernel_size=3,
                                padding=1))
        # 2) BatchNorm2d
        layers.append(nn.BatchNorm2d(num_features=out_feat))
        # 3) ReLU
        layers.append(nn.ReLU(inplace=True))
        
        if max_pool:
            layers.append(nn.MaxPool2d(3,2, padding=1))
        super().__init__(*layers)

In [ ]:
# Comprobacion
my_logger.test("conv layer")


l = conv_layer(1,16)

elems = list(l.children())
assert isinstance(elems[0],nn.Conv2d), "primero conv2d"
assert isinstance(elems[1], nn.BatchNorm2d), "primero conv2d"
assert isinstance(elems[2], nn.ReLU), "primero conv2d"
assert elems[0].in_channels == 1, "nuermo de canales de entrada mal"
assert elems[0].out_channels == 16, "nuermo de canales de entrada mal"
my_logger.success("conv layer",3)



In [ ]:
# Modulo final antes de la salida, no tiene parametros por tanto no hace falta init
class Flatten(nn.Module):
    def forward(self, x):
        return torch.flatten(x, start_dim=1)

Ahora crearemos la red convolucional con los modulos anteriores, a estructura es la siguiente (tomaremos numF=16):
* Capa1, conv_layer input_channels->numF, con maxpool
* Capa2, conv_layer numF -> 2* numF, con maxpool
* Capa3, conv_layer 2*numF -> 4* numF, con maxpool
* Capa4, conv_layer 4*numF -> 4* numF, **sin** maxpool
* Capa5, conv_layer 4*numF -> 4* numF, **sin** maxpool
* Maxpool temporal (se proporciona)
* Dropout (se proporciona)
* Flatten (se proporciona)
* Lineal de salida (se proporciona)




In [ ]:
     
class MyNet(nn.Sequential):
    def __init__(self,dropoutProb = 0.2, num_classes = 10):
        input_channels = 1
        numF = 16
        # Tamaño del último maxPooling. Hace que la última capa solo tenga 1
        # neurona en la dimensión temporal
        timePoolSize = int(np.ceil(74/8))

        layers = []
        # añada las capas que se le indica en el texto
        layers.append(conv_layer(input_channels, numF, max_pool=True))

        layers.append(conv_layer(numF, 2 * numF, max_pool=True))

        layers.append(conv_layer(2 * numF, 4 * numF, max_pool=True))

        layers.append(conv_layer(4 * numF, 4 * numF, max_pool=False))

        layers.append(conv_layer(4 * numF, 4 * numF, max_pool=False))


        layers.append(nn.MaxPool2d((1,timePoolSize)))  
        layers.append(nn.Dropout2d(dropoutProb))
        layers.append(Flatten())
        layers.append(nn.Linear(64*4,num_classes)) #obtiene los logits
        

        super().__init__(*layers)
        

In [ ]:
my_logger.test("conv net")

X_test = torch.randn(1,1,32,74)
net = MyNet()
y = net(X_test)
assert y.shape == (1,10), "tendremos una salida"
assert len(list(net.modules()))==28, "revise la estructura de la red"
assert sum([len(p) for p in net.parameters()]) == 980, "revise la estructura de la red"
my_logger.success("conv net",3)



## Training Loop

Una vez definida la red la parte de entrenamiento es exactamente igual que la de la practica 5. 

En el caso de las redes convolucionales la GPU si está disponible supone una gran diferencia para entrenar los modelos. 

Comente la linea 'cpu' o 'cuda' y compruebe la diferencia abismal entre los tiempos de ejecución:

In [ ]:
from train_net import train
import torch.optim as optim
import time

#device = 'cpu'
device = 'cuda'

# Define learning rate
learning_rate = 0.001
batch_size = 512

# Define network
red = MyNet()

# Create Adam optimizer
#optimizer = optim.SGD(red.parameters(), lr=learning_rate, momentum =0.9)
optimizer = optim.Adam(red.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


num_epochs = 30
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

start_time = time.time()
train_losses, val_losses, train_acc, val_acc = train(red, 
                                                     loss_fn, 
                                                     train_dataloader,val_dataloader, 
                                                     optimizer, num_epochs, device=device)

print(f"Train finished in {time.time()- start_time} seconds") 


fig, axes = plt.subplots(2,1)
axes[0].plot(train_losses, label='train loss')
axes[0].plot(val_losses, label = 'val_loss')
axes[0].legend()
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(train_acc, label='train_acc')
axes[1].plot(val_acc, label='val acc')
axes[1].legend()
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('Accuracy')

## Prueba con grabación

Tomaremos el modelo del apartado anterior para hacer nuestras pruebas con audios grabados
al instante.



In [ ]:
from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio
import librosa

import matplotlib.pyplot as plt


* Ejecuta la siguiente celda.
* Pulsa circulo negro para comenzar a grabar.
* Di un numero
* Pulsa de nuevo para terminar la grabacion

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

In [ ]:
# convertimos a wav
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -ar 16000 -f wav file.wav  -y -hide_banner -loglevel panic

In [ ]:
# Escucha para comprobar que todo sigue ok
Audio('file.wav')

In [ ]:
# Pintamos para ver el aspecto de la señal  que no sature 
audio_data, fs = librosa.load('file.wav', sr=None)
t = np.arange(len(audio_data))/fs
plt.plot(t,audio_data)
_ = plt.xlabel('time')

In [ ]:
S =  preprocess_audio('file.wav')

librosa.display.specshow(S, sr=16000, hop_length=240, x_axis='time', y_axis='mel')

In [ ]:
red.eval()

x = torch.from_numpy(S.astype(np.float32))
x = x.unsqueeze(0).unsqueeze(0) # añadimos dimensiones para poder hacer la inferencia
x = x.to(device)
y = red(x)
print("Has dicho: ", torch.argmax(y).item()) # clasificamos con el máximo

In [ ]:
import torch.nn.functional as F

In [ ]:
plt.bar(np.arange(10),F.softmax(y,dim=-1).cpu().detach().numpy().flatten())
plt.xlabel('Numero')
plt.ylabel('Probabilidad')
